# ITM 891 - PROJECT ITINERARY GENERATOR

The project aims to generate an itinerary for anyone who plans to visit London. The itinerary is generated based on several business rules.The notebook is divided into several parts, covering the various processes that was gone into generating the itinerary.

Part 1: Data Collection

Part 2: Data Cleaning and Processing

Part 3: Generate Itinerary

Part 4: Calculate score for futue expansion


## Part 1: Data Collection

Data is collecetd from 2 sources. 
The tourist sites data is collected from TripAdvisor and the London Tube data is taken from Wikipedia.

## Part 1 (a): Tourist Site Data

Below is the code to collect the data from TripAdvisor using Selenium
The source links for different categories of tourist sites is as follows,
Museums:https://www.tripadvisor.com/Attractions-g186338-Activities-c49-London_England.html

History Museums:https://www.tripadvisor.com/Attractions-g186338-Activities-c49-t30-London_England.html#FILTERED_LIST

Street Markets:https://www.tripadvisor.com/Attractions-g186338-Activities-c26-t142-London_England.html

Breweries:https://www.tripadvisor.com/Attractions-g186338-Activities-c36-t133-London_England.html

Historic Sites :https://www.tripadvisor.com/Attractions-g186338-Activities-c47-t17-London_England.html

LandMarks: https://www.tripadvisor.com/Attractions-g186338-Activities-c47-London_England.html

Parks: https://www.tripadvisor.com/Attractions-g186338-Activities-c57-London_England.html

Architectural building : https://www.tripadvisor.com/Attractions-g186338-Activities-c47-t3-London_England.html

Theatre: https://www.tripadvisor.com/Attractions-g186338-Activities-c58-t111,116-London_England.html

Art Galleries:https://www.tripadvisor.com/Attractions-g186338-Activities-c49-t28-London_England.html

All the data files have been stored as CSV and attached in the project submission folder


In [3]:
#Import Libraries
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [101]:
#initiating driver and lauching the required page
driver = webdriver.Chrome(executable_path='C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/chromedriver.exe')
#Launch the required link
driver.get("https://www.tripadvisor.com/Attractions-g186338-Activities-c49-t28-London_England.html")
# This is for the driver to implicity wait until it finds the element. That way, loading time is covered.
driver.implicitly_wait(20)

In [ ]:
#Get the total no of pages present at the bottom for each category.
try:
    pages=(driver.find_elements_by_xpath("//a[contains(@class, 'pageNum taLnk')]")[-1]).text
except Exception as e:
    pages=1
pages

In [103]:
# Create an empty Data Frame
Tourist_dict={}

In [93]:
# Incase driver is interrupted, move back to parent window. Do not run otherwise
driver.switch_to_window(window_before)

C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [104]:
'''
In this cell, after launching the page for a category, the driver clicks on individual site links and gathers 
required information. The driver switches control back and forth between parent and child window.
A dictionary is used to store the data of tourist sites.
Key -> Title of tourist site
Value -> A list containing other information pertaining to the site, in the order of
Suggested Duration
Address
Rating
Transport

The driver navigates till the last page of that particular category.
Common Exceptions are handled.

Since the data in TripAdvisor is arranged in decreasing order of weighted rating (ie rating value and number of ratings),
the final data frame to which data is stored also has Tourist sites arranged in decreasing order of the weighted ratings.
Hence the site picked from top of the data frame, always has higher rating in that category.

'''
#Iterate over each page
for i in range(1,int(pages)+1):
    Page_titles=[]
    elements=driver.find_elements_by_xpath("//div[@class='listing_title  title_with_snippets ']")

    # Switch window and collect other relavent information
    for element in elements:
        if(element.text not in list(Tourist_dict.keys())):
            # Get info such as Title, Overview, Duration, Address, Transport
            Title=''
            info_list=[]
            #get title
            Title=element.text
            element.click()
            window_before = driver.window_handles[0]
            window_after = driver.window_handles[1]
            #switch windows to child window
            driver.switch_to.window(window_after)
            #Get Duration    
            try:
                about_card=[]
                about_card=driver.find_elements_by_xpath("//div[contains(@class,'attractions-attraction-detail-about-card')]")
                flag=0
                for val in about_card:
                    if('Suggested Duration' in val.text):
                        info_list.append(val.text)
                        flag=1
                        break
                if(flag == 0):
                        info_list.append('Suggested Duration Not Available')
            #Handle no element exceptions
            except NoSuchElementException as e:
                info_list.append('Suggested Duration Not Available')
            # Get Address     
            try:
                info_list.append(driver.find_element_by_xpath("(//div[contains(@class,'attractions-contact-card-ContactCard')])[2]").text)
            except NoSuchElementException as e:
                info_list.append('Address Not Available')
            # Get Ratings
            try:
                info_list.append(driver.find_element_by_xpath("//a[@class='_1d_R5B7y']").text)
            except NoSuchElementException as e:
                info_list.append('Ratings Not Available')
            # Get Transport    
            try:
                trans=driver.find_elements_by_xpath("//span[contains(@class,'attractions-attraction-review-location-gettingthere-NearbyTransport__name')]")
                transport_list=[]
                for val in trans:
                    transport_list.append(val.text)
                info_list.append(transport_list) 
            except NoSuchElementException as e:
                info_list.append("Transport Info Not Available")
                
            Tourist_dict[Title]=info_list
            # Close the window and switch to parent window
            driver. close()
            driver.switch_to_window(window_before)
    try:    
        driver.find_element_by_xpath("//a[contains(text(),'Next')]").click()
    except NoSuchElementException as e:
        print("Reached the Last Page.")
    
   

C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DeprecationWarning: use driver.switch_to.window instead


Reached the Last Page.


In [105]:
'''
The key and vlaues from the previously generated dictionary are stored in separate lists, to later combine it to a master 
data frame for each category
'''

title=[]
Duration=[]
Address=[]
Transport=[]
Ratings=[]

for site in Tourist_dict:
    title.append(site)
    Duration.append(Tourist_dict[site][0])
    Address.append(Tourist_dict[site][1])
    Ratings.append(Tourist_dict[site][2])
    Transport.append(Tourist_dict[site][3])

In [106]:
#Create a dataframe for sites, for different categories.
Tourist_Sites = pd.DataFrame(list(zip(title, Duration, Address, Ratings, Transport)), columns=['Title', 'Suggested Duration','Address','Ratings','Transport']) 

In [107]:
Tourist_Sites.tail()

,Title,Suggested Duration,Address,Ratings,Transport
106,Crosby Hall,Suggested Duration Not Available,"Crosby Hall Cheyne Walk, London SW3 5AZ England",Ratings Not Available,"[Southgate, Oakwood, Rental Cars]"
107,Millbank Tower,Suggested Duration Not Available,"21-24 Millbank, London England",Ratings Not Available,"[Waterloo, Embankment, Rental Cars]"
108,Merchant Taylors' Hall,Suggested Duration Not Available,"Merchant Taylors Hall 30 Threadneedle Street, ...",Ratings Not Available,"[Bank, Bank, Rental Cars]"
109,Swiss Cottage Library,Suggested Duration Not Available,"88 Avenue Road London NW3 3HA, London NW3 3HA ...",Ratings Not Available,"[Swiss Cottage, Finchley Road, Rental Cars]"
110,Architectural Association School of Architecture,Suggested Duration Not Available,"36 Bedford Square, London WC1B 3ES England",Ratings Not Available,"[Tottenham Court Road, Goodge St, Rental Cars]"


In [108]:
# Save as a CSV file for each category
Tourist_Sites.to_csv("Architechture.csv")

## Part 1 (b) : Scraping London Tube Data

In [70]:
from bs4 import BeautifulSoup
import urllib
import csv

## Data Source:

Following are the wiki pages with the stations for different lines

BakerLoo: https://en.wikipedia.org/wiki/Bakerloo_line

Central: https://en.wikipedia.org/wiki/Central_line_(London_Underground)

Circle: https://en.wikipedia.org/wiki/Circle_line_(London_Underground)

District: https://en.wikipedia.org/wiki/District_line

DLR: https://en.wikipedia.org/wiki/List_of_Docklands_Light_Railway_stations

Hammersmith & City : https://en.wikipedia.org/wiki/Hammersmith_%26_City_line

Jubliee: https://en.wikipedia.org/wiki/Jubilee_line

Metropolitan: https://en.wikipedia.org/wiki/Metropolitan_line

Northern : https://en.wikipedia.org/wiki/Northern_line

Multiple tables present for different sub branches

Table 0 : Barnet

Table 1 : Edgware

Table 2 : Camden (not needed)

Table 3 : Charing

Table 4 : Bank

Table 5 : Main


Piccadilly : https://en.wikipedia.org/wiki/Piccadilly_line

Victoria: https://en.wikipedia.org/wiki/Victoria_line

The stations are collected and stored in a pickle file. Please run immediate cell below to get the stations.
Subsequent cells is the code that scrapes data from indiviual tube line pages.


In [ ]:
#Run to get the Stations_dict
import pickle
Stations_dict=pickle.load( open( "file.pkl", "rb" ) )

### The code below was used to scrape data from Wiki page of each tube line

In [ ]:
Stations_dict={}

In [153]:
source_url = 'https://en.wikipedia.org/wiki/Victoria_line'
with urllib.request.urlopen(source_url) as response:
    soup = BeautifulSoup(response)

In [154]:
#Scrape the stations here
# For most lines, this cell can be used to scrape data
rows=soup.find( "table", {"class":"wikitable sortable"} ).find('tbody').findAll('tr')
stations=[]
for row in rows:
    if len(row.findAll('td')) >0 :
        stations.append(row.findAll('td')[0].text.strip())
stations

['Walthamstow Central',
 'Blackhorse Road',
 'Tottenham Hale',
 'Seven Sisters   [SS]',
 'Finsbury Park',
 'Highbury & Islington',
 "King's Cross St. Pancras",
 'Euston',
 'Warren Street',
 'Oxford Circus',
 'Green Park',
 'Victoria  ( Trains to Gatwick)',
 'Pimlico',
 'Vauxhall',
 'Stockwell',
 'Brixton']

In [155]:
# Add stations to the dictionary
Stations_dict['Victoria']=stations
Stations_dict.keys()

dict_keys(['BakerLoo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith', 'Jubliee', 'Metropolitan', 'Northern_Barnet', 'Northern_Edgware', 'Northern_Charing', 'Northern_Bank', 'Northern_Main', 'Piccadilly', 'Victoria'])

In [147]:
#For Northen Line scrape for multiple tables by changing the number within the square brackets
#Scrape the stations here
rows=soup.find_all( "table", {"class":"wikitable sortable"} )[5].find('tbody').findAll('tr')
stations=[]
for row in rows:
    if len(row.findAll('td')) >0 :
        stations.append(row.findAll('td')[0].text.strip())
stations

['Oval',
 'Stockwell',
 'Clapham North',
 'Clapham Common',
 'Clapham South',
 'Balham',
 'Tooting Bec',
 'Tooting Broadway',
 'Colliers Wood',
 'South Wimbledon',
 'Morden']

In [148]:
Stations_dict['Northern_Main']=stations
Stations_dict.keys()

dict_keys(['BakerLoo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith', 'Jubliee', 'Metropolitan', 'Northern_Barnet', 'Northern_Edgware', 'Northern_Charing', 'Northern_Bank', 'Northern_Main'])

In [150]:
# For Piccadilly the path for table is diff. 
rows=soup.find_all( "table", {"class":"wikitable"} )[3].find('tbody').findAll('tr')
stations=[]
for row in rows:
    if len(row.findAll('td')) >0 :
        stations.append(row.findAll('td')[0].text.strip())
stations

['Continuing from Acton Town',
 'Ealing Common',
 'North Ealing',
 'Park Royal',
 'Alperton',
 'Sudbury Town',
 'Sudbury Hill ( Sudbury Hill Harrow)',
 'South Harrow',
 'Rayners Lane',
 'Eastcote',
 'Ruislip Manor',
 'Ruislip',
 'Ickenham',
 'Hillingdon',
 'Uxbridge']

In [151]:
Stations_dict['Piccadilly']=stations
Stations_dict.keys()

dict_keys(['BakerLoo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith', 'Jubliee', 'Metropolitan', 'Northern_Barnet', 'Northern_Edgware', 'Northern_Charing', 'Northern_Bank', 'Northern_Main', 'Piccadilly'])

In [ ]:
#Remove brackets [],() and text within those brackets, to get clear texts.
# examples: convert text from Sudbury Hill ( Sudbury Hill Harrow) to Sudbury Hill

stations=Stations_dict['Victoria']
#Only for JUbliee remove take only 27 stations. Last observation is a sentance and not a station
#stations=Stations_dict['Jubliee'][0:27]
j_stations=[]
for val in stations:
    try:
        index=val.index('[')
        val=val[0:index]
    except ValueError:
        print()
    try:
        index=val.index('(')
        val=val[0:index]
    except ValueError:
        print()
        
    j_stations.append(val.strip())
Stations_dict['Victoria']=j_stations

In [160]:
#RUN only once
#Store is as a pickcle to access it later
f = open("file.pkl","wb")
pickle.dump(Stations_dict,f)
f.close()

In [158]:
#Run to get the Stations_dict
Stations_dict=pickle.load( open( "file.pkl", "rb" ) )

In [159]:
Stations_dict

{'BakerLoo': ['Harrow & Wealdstone',
  'Kenton',
  'South Kenton',
  'North Wembley',
  'Wembley Central',
  'Stonebridge Park',
  'Harlesden',
  'Willesden Junction',
  'Kensal Green',
  "Queen's Park",
  'Kilburn Park',
  'Maida Vale',
  'Warwick Avenue',
  'Paddington  ( Trains to Heathrow)',
  'Edgware Road',
  'Marylebone',
  'Baker Street',
  "Regent's Park",
  'Oxford Circus',
  'Piccadilly Circus',
  'Charing Cross',
  'Embankment',
  'Waterloo',
  'Lambeth North',
  'Elephant & Castle'],
 'Central': ['West Ruislip',
  'Ruislip Gardens',
  'South Ruislip',
  'Northolt',
  'Greenford',
  'Perivale',
  'Hanger Lane',
  'Ealing Broadway',
  'West Acton',
  'North Acton',
  'East Acton',
  'White City',
  "Shepherd's Bush",
  'Holland Park',
  'Notting Hill Gate',
  'Queensway',
  'Lancaster Gate',
  'Marble Arch',
  'Bond Street',
  'Oxford Circus',
  'Tottenham Court Road',
  'Holborn',
  'Chancery Lane',
  "St. Paul's",
  'Bank',
  'Liverpool Street',
  'Bethnal Green',
  'Mile 

## Part 2 : Data Cleaning and Processing

This is the part where the different datasets that were created for different categories in Part 1 are taken and certain actions are performed on them. Following are the things done
1. Remove repeated values
2. Add a column called Tags to each site
3. Remove rental cars from the transport column of each site
4. Convert Suggested Duration column from string to float (in hrs)

In [4]:
#Read the data from the csv files. Change the path here
Museums=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/Museums.csv')
Architechture=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/Architechture.csv')
Brewery=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/Brewery.csv')
Historic=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/HistoricSites.csv')
HistoricMuseums=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/HistoryMuseums.csv')
LandMarks=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/LandMarks.csv')
Parks=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/Parks.csv')
StreetMarkets=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/StreetMarkets.csv')
Theatre=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/Theatre.csv')
ArtGalleries=pd.read_csv('C:/Users/Tina/Desktop/Spl Topic for Info Mgt/Ex/HopStop/ArtGalleries.csv')

In [ ]:
# Checking the dimensions of data frames before adding Museum tags
print(f' Shape of Museums is {Museums.shape}')
print(f' Shape of Arch is {Architechture.shape}')
print(f' Shape of Historic is {Historic.shape}')
print(f' Shape of LandMarks is {LandMarks.shape}')
print(f' Shape of Parks is {Parks.shape}')
print(f' Shape of StreetMarkets is {StreetMarkets.shape}')
print(f' Shape of Theatre is {Theatre.shape}')
print(f' Shape of ArtGalleries is {ArtGalleries.shape}')
print(f' Shape of Brewery is {Brewery.shape}')

### Common methods

These are the common methods used in subsequent cells.

In [8]:
#Function to find the intersection of 2 lists
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

# This is to search for some keywords and add appropriate tags in a data frame        
def search_add(df,search_key, tag):
    for i in range (0,len(df['Title'])):
        try:
            if df['Title'][i].lower().find(search_key.lower()) > -1 :
                if tag not in df['Tags']:
                    df['Tags'][i].append(tag)
        except KeyError:
            next
            
# This method searches for common titles between two data frames and tags to the first data frame that is passed to it
def add_tags(df1, df2, tag):
    Common_list=intersection(list(df1['Title']), list(df2['Title'])) 
    for i in range (0,len(df1['Title'])):
        try:
            if df1['Title'][i] in Common_list:
                if tag not in df1['Tags']:
                    df1['Tags'][i].append(tag)
        except KeyError:
            next

#This function is to check the no of obeservation that have a particular tag
def count_tag(df, tag):
    count=0
    for val in df['Tags']:
        if tag in val:
            count=count+1
    return count

#Convert transport column to a list and remove rentals from that
def remove_rental (df):
    for val in range (0,len(df['Transport'])):
        try:
            if(type(df['Transport'][val])==str):
                df['Transport'][val]=df['Transport'][val].replace("'","").replace("[","").replace("]","").replace(" ","").split(",")
                if('RentalCars' in df['Transport'][val]):
                    df['Transport'][val].remove('RentalCars')
        except KeyError:
            next
            
# Lets change all the suggested duration data type to numbers
def duration_to_float(df):
    for i in range (0,df.shape[0] ):
        try:
            # Assigning number of hours to observation where no of hours is not present
            if df['Suggested Duration'][i].find('Not') == -1:
                #If Theatre then make it 3 hrs
                if "Theatre" in df['Tags'][i]:
                    df['Suggested Duration'][i]=3
                    next
                else:
                    df['Suggested Duration'][i]=1.5
                    next
            elif df['Suggested Duration'][i].find('<'):
                    df['Suggested Duration'][i]=1
            else:
                hours=df['Suggested Duration'][i].replace("Suggested Duration:","").replace("hours","").split("-")
                df['Suggested Duration'][i]=float(hours[1])
                
                
        except KeyError:
            next

## Tagging

Tagging is done using the method add_tags (present in the common methods)

A tag, is a list that indcates the different category a site belongs to

In [ ]:
# Add a Museums tag column and enter museum in it
Museums['Tags']=[['Museum'] for x in range(len(Museums))]
LandMarks['Tags']=[['LandMark'] for x in range(len(LandMarks))]
Brewery['Tags']=[['Brewery'] for x in range(len(Brewery))]
StreetMarkets['Tags']=[['StreetMarket'] for x in range(len(StreetMarkets))]
Theatre['Tags']=[['Theatre'] for x in range(len(Theatre))]
Parks['Tags']=[['Park'] for x in range(len(Parks))]
Historic['Tags']=[['History'] for x in range(len(Historic))]
Architechture['Tags']=[['Architechture'] for x in range(len(Architechture))]
ArtGalleries['Tags']=[['Art'] for x in range(len(ArtGalleries))]

In [10]:
#Lets add other tags to observation in the LandMarks dataframe
add_tags(LandMarks, Museums, 'Museum')
add_tags(LandMarks, Historic, 'History')
add_tags(LandMarks, Architechture, 'Architechture')
add_tags(LandMarks, Parks, 'Park')
add_tags(LandMarks, StreetMarkets, 'StreetMarket')
add_tags(LandMarks, Theatre, 'Theatre')
add_tags(LandMarks, ArtGalleries, 'Art')
add_tags(LandMarks, Brewery, 'Brewery')

In [11]:
# #Lets add other tags to observation in the Museum dataframe
add_tags(Museums, LandMarks, 'LandMark')
add_tags(Museums, Historic, 'History')
add_tags(Museums, Architechture, 'Architechture')
add_tags(Museums, Parks, 'Park')
add_tags(Museums, StreetMarkets, 'StreetMarket')
add_tags(Museums, Theatre, 'Theatre')
add_tags(Museums, ArtGalleries, 'Art')
add_tags(Museums, Brewery, 'Brewery')

In [12]:
# History 
add_tags(Historic, LandMarks, 'LandMark')
add_tags(Historic, Architechture, 'Architechture')
add_tags(Historic, Parks, 'Park')
add_tags(Historic, StreetMarkets, 'StreetMarket')
add_tags(Historic, Theatre, 'Theatre')
add_tags(Historic, ArtGalleries, 'Art')
add_tags(Historic, Museums, 'Museum')
add_tags(Historic, Brewery, 'Brewery')

In [14]:
#Architechture
add_tags(Architechture, LandMarks, 'LandMark')
add_tags(Architechture, Historic, 'History')
add_tags(Architechture, Museums, 'Museum')
add_tags(Architechture, Parks, 'Park')
add_tags(Architechture, StreetMarkets, 'StreetMarket')
add_tags(Architechture, ArtGalleries, 'Art')
add_tags(Architechture, Theatre, 'Theatre')
add_tags(Architechture, Brewery, 'Brewery')

In [13]:
#Theatre
add_tags(Theatre, LandMarks, 'LandMark')
add_tags(Theatre, Architechture, 'Architechture')
add_tags(Theatre, Historic, 'History')
add_tags(Theatre, Museums, 'Museum')
add_tags(Theatre, Parks, 'Park')
add_tags(Theatre, StreetMarkets, 'StreetMarket')
add_tags(Theatre, Brewery, 'Brewery')
add_tags(Theatre, ArtGalleries, 'Art')

In [15]:
#Park
add_tags(Parks, LandMarks, 'LandMark')
add_tags(Parks, Architechture, 'Architechture')
add_tags(Parks, Historic, 'History')
add_tags(Parks, Museums, 'Museum')
add_tags(Theatre, Theatre, 'Theatre')
add_tags(Theatre, StreetMarkets, 'StreetMarket')
add_tags(Theatre, Brewery, 'Brewery')
add_tags(Theatre, ArtGalleries, 'Art')

In [16]:
#Art
add_tags(ArtGalleries, LandMarks, 'LandMark')
add_tags(ArtGalleries, Historic, 'History')
add_tags(ArtGalleries, Museums, 'Museum')
add_tags(ArtGalleries, Parks, 'Park')
add_tags(ArtGalleries, Architechture, 'Architechture')
add_tags(ArtGalleries, StreetMarkets, 'StreetMarket')
add_tags(ArtGalleries, Theatre, 'Theatre')
add_tags(ArtGalleries, Brewery, 'Brewery')

In [17]:
#Street Market
add_tags(StreetMarkets, LandMarks, 'LandMark')
add_tags(StreetMarkets, Historic, 'History')
add_tags(StreetMarkets, Museums, 'Museum')
add_tags(StreetMarkets, Parks, 'Park')
add_tags(StreetMarkets, Architechture, 'Architechture')
add_tags(StreetMarkets, ArtGalleries, 'Art')
add_tags(StreetMarkets, Theatre, 'Theatre')
add_tags(StreetMarkets, Brewery, 'Brewery')

## Create Sports Data Frame

The cell below was used to create a data frame for sports.
The selection was based on local knowledge and not logic

In [ ]:
#Add sports tag Wimbledon, Cricket, Stadium, Tennis, Football
search_add(Museums,'Stadium', 'Sports')
search_add(LandMarks,'Stadium', 'Sports')
search_add(Museums,'Wimbledon', 'Sports')
search_add(LandMarks,'Wimbledon', 'Sports')
search_add(Museums,'Cricket', 'Sports')
search_add(LandMarks,'Cricket', 'Sports')
search_add(Museums,'Tennis', 'Sports')
search_add(LandMarks,'Tennis', 'Sports')
search_add(Museums,'Football', 'Sports')
search_add(LandMarks,'Football', 'Sports')
#Create Sports DataFrame 
Sports=pd.DataFrame(columns = list(LandMarks.columns) )
for i in range(0,LandMarks.shape[0]):
    if ('Sports' in LandMarks['Tags'][i]):
        Sports=Sports.append(LandMarks.iloc[i,:], ignore_index=True)
for i in range(0,Museums.shape[0]):
    if ('Sports' in Museums['Tags'][i]):
        if (str(Museums['Title'][i]) not in list(Sports['Title'])):
            Sports=Sports.append(Museums.iloc[i,:], ignore_index=True)
Sports

## Create Art Data frame

While collecting data from TripAdvisor, not all information pertaining to art was collected.
Since Art is a complete subset of Museums, a separate dataframe for Art can be created using Museums dataframe

In [18]:
#Create Art DataFrame since the Art data collected from Trip Advisor does not have ALL the required information
Art=pd.DataFrame(columns = list(Museums.columns) )
for i in range(0,Museums.shape[0]):
    if ('Art' in Museums['Tags'][i]):
        Art=Art.append(Museums.iloc[i,:], ignore_index=True)

## Remove Rental Cars from Transport Column

The proximity calulation is based on the values present in the transport column of the data frame.
The transport column, has a list of stations near the tourist site.
Since ALL the columns have rental cars in that list, we are removing it.

In [19]:
#Lets remove all rental cars from Transport list
remove_rental(LandMarks)
remove_rental(Museums)
remove_rental(Historic)
remove_rental(Parks)
remove_rental(StreetMarkets)
remove_rental(Theatre)
remove_rental(Brewery)
remove_rental(Art)
remove_rental(Architechture)
remove_rental(Sports)

C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Convert Suggested Duration from string to float

The Suggested Duration is presently in the form
example -->'Suggested Duration is 2 hrs'

This is converted to 2, which is of data type float

In [20]:
# Lets change all the suggested duration to numbers
duration_to_float(LandMarks)
duration_to_float(Museums)
duration_to_float(Parks)
duration_to_float(StreetMarkets)
duration_to_float(Theatre)
duration_to_float(Historic)
duration_to_float(Art)
duration_to_float(Architechture)
duration_to_float(Sports)
duration_to_float(Brewery)

C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Tina\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Part 3 : Generating Itinerary

## Time

The python datetime package is used to obtain the following information from the inputs - start and end date passed.

Number of Days 

Season (Summer or Winter)

Season is calculated based on the location's seasonality

In [24]:
import datetime 
import calendar 
from datetime import timedelta  
# the function returns a list of daysofWeek and Season of the year
# Takes in input in %d %m %Y format
def findDays(start, end): 
    start_day=datetime.datetime.strptime(start, '%d %m %Y')
    end_day=datetime.datetime.strptime(end, '%d %m %Y')
    #Find no of days
    diff=end_day-start_day
    diff=int(str(diff).replace(" days, 0:00:00",""))
    Days=[str(calendar.day_name[start_day.weekday()])]
    next_day=start_day
    for days in range(0,diff):
        next_day=next_day+timedelta(days=1)
        Days.append(str(calendar.day_name[next_day.weekday()]))
    month= datetime.datetime.strptime(start, '%d %m %Y').month
    # Summer and Srping together considered as Summer
    if(month>= 2 & month<=7):
        Season='Summer'
    else:
        Season='Winter'
    return (Days,Season)
    
# Driver program 
date = '03 02 2019'
end = '05 02 2019'
findDays(date,end)

(['Sunday', 'Monday', 'Tuesday'], 'Summer')

## Proximity

Proximity is solely based on the Tube connectivity in London.
London is a major city, that is connected primarily by under ground tube. Hence the tube stations data give a fair judgement on proximity of different tourist sites

In [25]:
#This method finds a tourist site within immediate viccinity first.
#If it is not able to find with the same stations of a site, it seraches 2 stations away in each diretion of the site on different tube lines
#If it is not able to find within 2 stations of a site, it seraches 7 stations away in each diretion of the site on different tube lines
def find_nearby(df, site, Sites, Day_of_week):
    #site=site.to_frame()
    transport=site['Transport'][0]
    # Fisrt check for a site in immediate viccinity
    for i in range(0,df.shape[0]):
        if len(intersection(df['Transport'][i], transport)) > 0:
            # This condition is to avoid suggestion of Museums of a Weekday
            if (Day_of_week in ('Monday','Tuesday','Wednesday','Thursday','Friday') and 'Museum' in df['Tags'][i]):
                next
            # Check if it not present in the exisiting list already
            if not(str(df['Title'][i]) in list(Sites['Title'])):
                return df.iloc[i,:]
            else:
                next
    #Now check for sites that are accessible by tube with two stations distance
    by_tube=[]
    for tr in transport:
        for key, val in Stations_dict.items():
            if tr in val:
                index=val.index(tr)
                # Get pre two stations
                try:
                    by_tube.append(val[index-1])
                    by_tube.append(val[index-2])
                except:
                    None
                # Get next two stations
                try:
                    by_tube.append(val[index+1])
                    by_tube.append(val[index+2])
                except:
                    None
    for i in range(0,df.shape[0]):
        if len(intersection(df['Transport'][i], by_tube)) > 0:
            if (Day_of_week in ('Monday','Tuesday','Wednesday','Thursday','Friday') and 'Museum' in df['Tags'][i]):
                next
            # Check if it not present in the exisiting list already
            if not(str(df['Title'][i]) in list(Sites['Title'])):
                return df.iloc[i,:]
            else:
                next
    #Now check for sites that are accessible by tube with 5 stations distance
    by_tube=[]
    for tr in transport:
        for key, val in Stations_dict.items():
            if tr in val:
                index=val.index(tr)
                # Get pre 5 stations
                try:
                    by_tube.append(val[index-3])
                    by_tube.append(val[index-4])
                    by_tube.append(val[index-5])
                    by_tube.append(val[index-6])
                    by_tube.append(val[index-7])

                except:
                    None
                # Get next 5 stations
                try:
                    by_tube.append(val[index+3])
                    by_tube.append(val[index+4])
                    by_tube.append(val[index+5])
                    by_tube.append(val[index+6])
                    by_tube.append(val[index+7])
                except:
                    None
    for i in range(0,df.shape[0]):
        if len(intersection(df['Transport'][i], by_tube)) > 0:
            if (Day_of_week in ('Monday','Tuesday','Wednesday','Thursday','Friday') and 'Museum' in df['Tags'][i]):
                next
            # Check if it not present in the exisiting list already
            if not(str(df['Title'][i]) in list(Sites['Title'])):
                return df.iloc[i,:]
            else:
                next
        

## Bias

Bias determines the order in which categories are chosen while suggesting tourist sites. The diversity of each caetgory is calculated. Here diversity refers to how much of a category includes sites that belong to other category as well.

In [51]:
Excl_Rank_List=['LandMark','Museum','History','Architechture','Art','Theatre','Park','Sports','Market','Brewery']
cat_dict={'LandMark':LandMarks,'Museum':Museums,'History':Historic,'Architechture':Architechture,'Art':Art,'Theatre':Theatre,'Park':Parks,'Sports':Sports,'Market':StreetMarkets,'Brewery':Brewery}
# this method is to calculate the % of all categories in one particular category
# eg for Park, check the number of titles with tag of each category besides Park and divide by the total of that category
#the fraction caluculated for categories are added up to get total score of that category
def cal_bias():
    score_dict={}
    for key in cat_dict.keys():
        score=0
        for key_other in cat_dict.keys():
            if str(key)==str(key_other):
                next
            else:
                #calculate the % of all categories in one category
                score=score+count_tag(cat_dict[key],key_other)/cat_dict[key_other].shape[0]
            
        score_dict[key]=score
    return(score_dict)
        
        
cal_bias()    

{'LandMark': 2.8899179750225072,
 'Museum': 1.7044454891966416,
 'History': 0.6020732463971519,
 'Architechture': 0.40471216929240145,
 'Art': 0.07010140494713997,
 'Theatre': 0.06353229347784702,
 'Park': 0.060573908849770916,
 'Sports': 0.029120607160534565,
 'Market': 0.0027223230490018148,
 'Brewery': 0.0}

In [26]:
# returns the list of sites in ordered form based on their Bias
def ordered_list(tags):
    Excl_Rank_List=['LandMark','Museum','History','Architechture','Art','Theatre','Park','Sports','Market','Brewery']
    order=[]
    re_list=[]
    for i in tags:
        order.append(Excl_Rank_List.index(i))
    order.sort(reverse=True)
    for i in order:
        re_list.append(Excl_Rank_List[i])
    return re_list


['Sports', 'Theatre', 'Art', 'History', 'LandMark']

## Itinerary

In [62]:
"""
    The method returns an Itineraray as a dataframe.The dataframe has the following information about the tourist site.
    
    Arguments:
        start_date: The start date of the vacation in %d %m %Y format
        end_date: The end date of the vacation in %d %m %Y format    
        
    Returns:
        A data frame with the following columns
        Title
        Suggested Duration
        Address
        Ratings
        Transport
        Tags
        Day
        
    """
def generate_itinerary(start_date, end_date,tags):
    #Get number of days and season
    Days,Season=findDays(start_date,end_date)
    No_of_days=len(Days)
    
    #Initiate other variables
    Accounted_days=0
    Sites = pd.DataFrame(columns = list(LandMarks.columns) )
    Day=[]
    Total_hours=9
    #get List based on rank
    tags=ordered_list(tags)
    #Dictionary to hole category name and its corresponding df
    cat_dict={'LandMark':LandMarks,'Museum':Museums,'History':Historic,'Architechture':Architechture,'Art':Art,'Theatre':Theatre,'Park':Parks,'Sports':Sports,'Market':StreetMarkets,'Brewery':Brewery}
    for day in range(1,No_of_days+1):
        print(Days[day-1])
        Day_of_week=Days[day-1]
        Accounted_hours=0
        #pick the first item of the day. Biased towards exclusive one
        #This for loop is to start with exlusive tag, and in case no site is found we move to the next highly exclusive site to pick the first site of the day
        for tag in tags:
            df=cat_dict[tag] 
            for i in range(0, df.shape[0]):
                if not (str(df['Title'][i]) in list(Sites['Title'])):
                    if(Day_of_week in ('Monday','Tuesday','Wednesday','Thursday','Friday') and 'Museum' in df['Tags'][i]):
                        next
                    Sites=Sites.append(df.iloc[i,:], ignore_index=True)
                    #add day to the last observation of the day column
                    Day.append(day)
                    Accounted_hours=Accounted_hours+list(Sites['Suggested Duration'])[-1]+0.5
                    break
            # Exit if one site is found to begin the day        
            if(Sites.shape[0]>0):
                break

        while(Accounted_hours<Total_hours):
            for tag in tags:
                df=cat_dict[tag]
                #Initiate current site for every loop 
                current_site=pd.DataFrame(columns = list(Sites.columns) )
                current_site=current_site.append(Sites.iloc[-1,:], ignore_index=True)
                len1=Sites.shape[0]
                Sites=Sites.append(find_nearby(df,current_site, Sites,Day_of_week))
                len2=Sites.shape[0]
                # append day if there is a diff in len
                if (len2-len1>0):
                    Day.append(day)
                Accounted_hours=Accounted_hours+list(Sites['Suggested Duration'])[-1]+0.5
            #fitting if there are not enough things to do for the day    
            if (Accounted_hours<Total_hours):
                #Initiate current site for every loop 
                current_site=pd.DataFrame(columns = list(Sites.columns) )
                current_site=current_site.append(Sites.iloc[-1,:], ignore_index=True)
                len1=Sites.shape[0]
                Sites=Sites.append(find_nearby(LandMarks,current_site, Sites,Day_of_week))
                len2=Sites.shape[0]
                # append day if there is a diff in len
                if (len2-len1>0):
                    Day.append(day)
                Accounted_hours=Accounted_hours+list(Sites['Suggested Duration'])[-1]+0.5

        
        
    Sites['Day']=Day
    return (Sites)

## Sample Itinerary

Incase of no prefernce for user, pass only LandMark in the tags list.
Since LandMark is the category with highest number and variety, if a user does not give an interest, the tourist sites are chosen from LandMark category

In [169]:
Sites=generate_itinerary('01 02 2019','07 02 2019',['History'])
Sites

Friday
Saturday
Sunday
Monday
Tuesday
Wednesday
Thursday


,Unnamed: 0,Title,Suggested Duration,Address,Ratings,Transport,Tags,Day
0,0,Churchill War Rooms,1.5,"King Charles Street, Clive Steps Westminster, ...",4.5,"[CharingCross, Embankment]","[History, LandMark, Museum]",1
1,2,Houses of Parliament,1.5,"Parliament Square, London SW1A 0AA England",4.5,"[Waterloo, Embankment]","[History, LandMark, Architechture]",1
2,4,Westminster Abbey,1.5,"20 Dean's Yard Broad Sanctuary, London SW1P 3P...",4.5,"[CharingCross, Embankment]","[LandMark, History, Architechture]",1
3,16,Banqueting House,1.5,Whitehall Palace Whitehall Horse Guards Avenue...,4.5,"[CharingCross, Embankment]","[History, LandMark, Architechture, Museum]",1
4,10,London Eye,1.5,"Westminster Bridge Road Riverside Building, Co...",4.5,"[Waterloo, Embankment]",[LandMark],1
5,1,Tower of London,1.5,"St Katharine's & Wapping, London EC3N 4AB England",4.5,"[TowerHill, TowerGateway]","[History, LandMark]",2
6,8,HMS Belfast,1.0,"The Queen's Walk, London SE1 2JH England",4.5,"[LondonBridge, TowerHill]","[History, LandMark, Museum]",2
7,1,Tower Bridge,1.5,"Tower Bridge Road, London SE1 2UP England",4.5,"[GallionsReach, TowerHill]",[LandMark],2
8,73,The Undercroft Chapel,1.0,"Byward Street All Hallows By The Tower, London...",Ratings Not Available,"[TowerHill, TowerGateway]","[History, LandMark, Architechture]",2
9,21,St. Dunstan in the East,1.0,"St. Dunstan's Hill, London England",Ratings Not Available,"[Monument, TowerHill]",[LandMark],2


In [1132]:
#Save as a csv
Sites.to_csv("Itinerary.csv")

##  Part 4 : Scoring

Two scores are calculated for each day.

1. Goodness Score
2. Fit Score

Goodness score is calculated by the sum (site score in a tag given by user) / (no of items per day * no of tags given by user)
 
Fit score is calculated by the sum (site score in a tag not given by user) / (no of items per day * no of tags given by user)
 
site score is 1 if it is in top 25 % in an category

site score is 0.75 if it is between 25 % and 50 % in a category

site score is 1 if it is in between 50% and 75 % in a category

site score is 0.25 if it is in bottom 25 % in a category

The score gives a fair idea if the iteneraray has satistfied users interest with suggested hours and proximity factored in.

Since the data is arranged in decreasing order of weighted rating (ordered so while collecting ir from Trip Advisor)

For the future scope scope of the Project, the bias and fitting should be changed to make these scores better


In [170]:
"""
    The method calculates the daily score for the iteniraray generated
    
    Arguments:
        Sites: The sites dataframe  generated by the Itineraray generator
        Tags: The list of tags used by the generator to generate the itineraray above
        
    """
def score_cal(Sites,tags):
    day_score={}
    fit={}
    days=len(Sites['Day'].unique().tolist())
    for i in range (1,days+1):
        score=0
        fit_score=0
        #df of that day
        df=Sites[Sites['Day']==i].reset_index(drop=True)
        item_p_day=df.shape[0]
        for val in range(0,df.shape[0]):
            for tag in list(df['Tags'][val]):
                cat_df=cat_dict[tag]
                index=(cat_df[cat_df['Title']==str(df['Title'][val])].index.values[0])
                qtr=int(cat_df.shape[0]/4)
                #Calculate main score
                if tag in tags:
                    if (index< qtr):
                        score=score+1
                    elif(index>= qtr & index< (qtr*2)):
                        score=score+0.75
                    elif(index>= (qtr*2) & index< (qtr*3)):
                        score=score+0.5
                    else:
                        score=score+0.25 
                #calculate fit score
                else:
                    if (index< qtr):
                        fit_score=fit_score+1
                    elif(index>= qtr & index< (qtr*2)):
                        fit_score=fit_score+0.75
                    elif(index>= (qtr*2) & index< (qtr*3)):
                        fit_score=fit_score+0.5
                    else:
                        fit_score=fit_score+0.25
        dr=len(tags)*item_p_day
        score=score/dr
        fit_score=fit_score/dr
        day_score[i]=score
        fit[i]=fit_score
    print("Main Score")    
    print(day_score)
    print("Fit Score")
    print(fit)

In [167]:
score_cal(Sites,['History'])

Main Score
{1: 0.8, 2: 0.625, 3: 0.6, 4: 0.6, 5: 0.5416666666666666, 6: 0.6875, 7: 0.6666666666666666}
Fit Score
{1: 2.0, 2: 1.4166666666666667, 3: 1.2, 4: 1.6, 5: 1.0416666666666667, 6: 1.4375, 7: 1.6666666666666667}
